In [ ]:
import cv2
import numpy as np
import tempfile
import os

def extract_frames(
    video_file,
    num_frames=50,
    target_size=(224, 224)
):
    """
    Extracts uniformly sampled frames from a video file.

    Parameters
    ----------
    video_file : UploadedFile or str
        Streamlit uploaded file or path to video
    num_frames : int
        Number of frames to extract
    target_size : tuple
        Frame resize size (H, W)

    Returns
    -------
    np.ndarray
        Shape: (num_frames, H, W, 3)
    """

    # Handle Streamlit UploadedFile
    if hasattr(video_file, "read"):
        tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
        tmp.write(video_file.read())
        video_path = tmp.name
    else:
        video_path = video_file

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames == 0:
        cap.release()
        raise ValueError("Empty or unreadable video.")

    frame_indices = np.linspace(
        0, total_frames - 1, num_frames, dtype=int
    )

    frames = []
    idx_set = set(frame_indices)

    i = 0
    ret = True
    while ret:
        ret, frame = cap.read()
        if not ret:
            break

        if i in idx_set:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, target_size)
            frames.append(frame)

        i += 1

    cap.release()

    # Cleanup temp file
    if hasattr(video_file, "read"):
        os.unlink(video_path)

    if len(frames) < num_frames:
        raise ValueError("Not enough frames extracted from video.")

    return np.array(frames, dtype=np.float32)
